In [1]:
import pandas as pd
import configparser # for read config.ini
import sqlalchemy # for connect to db
from sqlalchemy import create_engine # for create engine

config = configparser.ConfigParser()
config.read('./config.ini')

['./config.ini']

In [4]:
config.get('SYBASE_PROD', 'dns')

'DWH_SYBASE_PROD'

In [8]:
## Need created ODBC connect x64 before start
def connected_bd(db):
    if db == 'ORA':
        user = config.get('ORA_DB', 'user')
        pwd = config.get('ORA_DB', 'pwd')
        dsn = cx_Oracle.makedsn(
            'dwh.ukr',
            '1525',
            service_name='dwh.ukr')
        conn_ora = cx_Oracle.connect(
            user=user,
            password=pwd,
            dsn=dsn)
        return conn_ora
    
    if db == 'sysprod':
        user = config.get('SYBASE_PROD', 'user')
        pwd = config.get('SYBASE_PROD', 'pwd')
        dns = config.get('SYBASE_PROD', 'dns')
        engine = create_engine(f"sybase+pyodbc://{user}:{pwd}@{dns}")
        return engine
    
    if db == 'systest':
        user = config.get('SYBASE_TEST', 'user')
        pwd = config.get('SYBASE_TEST', 'pwd')
        dns = config.get('SYBASE_TEST', 'dns')
        engine = create_engine(f"sybase+pyodbc://{user}:{pwd}@{dns}")
        return engine

In [9]:
engine_prod = connected_bd('sysprod')

In [17]:
ddate = "convert(datetime, '2022-09-30', 121)"


field = """a.CUSTOMER,
           a.CUST_LOAN_GROUP,
           a.DESCRIPTION,
           a.CREATE_PRINCIPLE,
           a.VALID_FROM,
           a.VALID_TO,
           a.MAIN_CUSTOMER,
           b.CUSTOMER_STATUS, 
           b.SHORT_NAME_LNG"""

exp = """,SUM( 
                db.DISCOUNT_EUR  
                + db.DEFF_EXPENSE_EUR 
                + db.DEFF_INCOME_EUR  
                + db.INTEREST_RES_EUR  
                + db.OVERDUE_RES_INTEREST_EUR  
                + db.INTEREST_PNLT_RES_EUR  
                + db.INTEREST_SP_PNLT_RES_EUR  
                + db.OVERDUE_SP_RES_INTEREST_EUR  
                + db.INTEREST_EUR  
                + db.INTEREST_SUSP_EUR  
                + db.OFF_BALANCE_EUR  
                + CASE WHEN (db.PRINCIPAL_EUR  
                + db.PART_WOFF_EUR  
                + db.OVERDUE_PRINCIPAL_EUR  
                + db.OTHER_CHARGES_EUR) > 0 THEN 0  
                ELSE (db.PRINCIPAL_EUR  
                + db.PART_WOFF_EUR  
                + db.OVERDUE_PRINCIPAL_EUR  
                + db.OTHER_CHARGES_EUR) END  
                ) AS EXPOSURE"""


query = f"""select {field} {exp}
                from  DWH_OWNER.D_FL_CUSTOMER_LOAN_GROUP a join DWH_OWNER.D_FL_CUSTOMER b 
             on a.CUSTOMER = b.CUSTOMER_CODE 
             and b.CUSTOMER_STATUS = 900 
             and b.VALID_TO > {ddate}
             and a.VALID_TO > {ddate}
             and a.CUSTOMER != '-1'
                left join DWH_OWNER.F_FL_DAILY_BALANCE db 
             on db.D_DATE = {ddate}
             and db.CUSTOMER = a.CUSTOMER
             AND db.APP_CODE != 'SC' 
             AND db.CATEGORY NOT IN (21074, 21076, 8016, 5000, 1150) 
             AND db.INF_POSITION = 'IFRS' 
             AND db.IMP_STAGE != '-1'
              group by {field} """


In [18]:
%%time
df = pd.read_sql(query, engine_prod)

CPU times: total: 46.9 ms
Wall time: 840 ms


In [19]:
df.head()

,CUSTOMER,CUST_LOAN_GROUP,DESCRIPTION,CREATE_PRINCIPLE,VALID_FROM,VALID_TO,MAIN_CUSTOMER,CUSTOMER_STATUS,SHORT_NAME_LNG,EXPOSURE
0,103239,10,ADG,1,2019-09-10,9999-01-01 00:00:00,-1,900,ADG GROUP SIA,NaN
1,107068,2808,Loputevs Aleksandrs,1,2019-09-10,9999-01-01 00:00:00,-1,900,ELIONS LA SIA,NaN
2,109437,2517,Voronkova Antonina,2,2019-09-10,9999-01-01 00:00:00,-1,900,VORONKOVA ANTOŅINA,-51032.38
3,1100736,617,MOOSUS,1,2019-09-10,9999-01-01 00:00:00,-1,900,MOOSUS MÄRT,NaN
4,1101199,5421,EfTEN Kinnisvarafond II group,1,2022-02-17,9999-01-01 00:00:00,-1,900,ASTLANDA HOTELLI AS,NaN


In [1]:
260384621.43 / 10**6

260.38462143